# 🥱 LangGraph_-Adaptive-RAG 篇

## 適應性檢索增強生成（Adaptive RAG）策略

適應性檢索增強生成（Adaptive RAG）是一種先進的檢索增強生成（RAG）策略，它結合了兩個關鍵元素：

1. 查詢分析
2. 主動式／自我修正式 RAG

根據相關研究論文，查詢分析可用於在以下三種模式間進行路由選擇：

* 無檢索模式
* 單次 RAG 模式
* 迭代式 RAG 模式

我們將利用 LangGraph 工具來擴展這個概念。在我們的實作中，我們將在以下幾種方式間進行路由：

* 網路搜尋：用於處理與近期事件相關的問題
* 自我修正式 RAG：用於處理與我們索引相關的問題

這種方法能夠根據不同類型的查詢，靈活地選擇最合適的處理方式，從而提高回答的準確性和相關性。

❤️ Created by [hengshiousheu](https://huggingface.co/Heng666).

![adaptive-rag.png](https://i.imgur.com/r7mXnu3.png)
[圖源取自 Langgraph 官網](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/)

# 環境設置
## 安裝必要套件
首先，我們需要安裝一些關鍵的 Python 套件：

In [ ]:
%pip install --upgrade --quiet langchain_community
%pip install --upgrade --quiet tiktoken
%pip install --upgrade --quiet langchain-openai
%pip install --upgrade --quiet langchainhub
%pip install --upgrade --quiet chromadb
%pip install --upgrade --quiet langchain
%pip install --upgrade --quiet langgraph
%pip install --upgrade --quiet tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362

## 設置 API 金鑰
我們需要設置 環境變數 OPENAI_API_KEY ，可以直接完成，如下所示：

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

> 🔒 請注意保護好你的 API 金鑰，避免洩露或濫用。

### (可用可不用)LangSmith

你用LangChain構建的許多應用程式將包含多個步驟，並多次調用LLM調用。隨著這些應用程式變得越來越複雜，能夠檢查您的鏈或代理內部到底發生了什麼變得至關重要。最好的方法是與[LangSmith](https://smith.langchain.com)合作。

請注意，LangSmith 不是必需的，但它很有説明。如果您確實想使用 LangSmith，請在上面的鏈接中註冊后，請確保設置環境變數以開始記錄跟蹤：

In [ ]:
import os
import getpass
from datetime import datetime
import pytz

current_time = datetime.now(pytz.timezone('Asia/Taipei')).strftime("%Y-%m-%d %Z")

os.environ["LANGCHAIN_TRACING_V2"] = "false" ##想要使用記得改 true, 不要時改 false.
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = f"LangGraph Adaptive-RAG-{current_time}"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

## 建立向量資料庫

畢竟是 RAG ，你懂的。在此用記憶體資料庫 `Chroma` 協助我們完成範例。實際案例中，會依照專案不同選用不同的向量資料庫以及保存方式。

In [ ]:
### Build Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import LanceDB
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


###### router import

from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding_function = OpenAIEmbeddings()

urls = [
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

 # Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512, chunk_overlap=0
    )
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embedding_function
    )
retriever = vectorstore.as_retriever()

print("vectoselected",vectorstore)
retriever = vectorstore.as_retriever()

vectoselected <langchain_community.vectorstores.chroma.Chroma object at 0x787070d1a830>


這個步驟創建了一個包含相關文檔的向量資料庫，為後續的檢索操作做好準備。

## 構建系統組件

### 路由器（Router）

路由器用於決定是使用向量存儲還是網絡搜索來回答問題：

In [ ]:
### Build Index
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """你是一個專家，負責將用戶問題路由到向量存儲或網絡搜索。
向量存儲包含與代理、提示工程和對抗性攻擊相關的文檔。
對於這些主題的問題使用向量存儲。否則，使用網絡搜索。"""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router


question1 ="李多慧什麼時候加入統一獅？"
print(question_router.invoke({"question": question1}))
print(question_router.invoke({"question": "請問 Agent 是什麼？"}))


datasource='web_search'
datasource='vectorstore'


## 檢索評分器
這個組件用於評估檢索文檔與問題的相關性：

In [ ]:
### Retrieval Grader

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """你是一個評分員，評估檢索文檔與用戶問題的相關性。 \n
    如果文檔包含與用戶問題相關的關鍵詞或語義含義，將其評為相關。 \n
    這不需要是嚴格的測試。目的是過濾掉錯誤的檢索結果。 \n
    給出二元評分 'yes' 或 'no' 來表示文檔是否與問題相關。"""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "檢索文檔：\n\n {document} \n\n 用戶問題：{question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)

doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


binary_score='no'


## 模擬檢索成功的文檔

In [ ]:
class Document:
    def __init__(self, page_content):
        self.page_content = page_content

    def __repr__(self):
        return f"Document(page_content='{self.page_content}')"

cleaned_docs_format = [Document(doc.page_content) for doc in docs]

for doc in cleaned_docs_format:
    print(doc)

## 生成節點
用途：生成節點負責基於檢索的文檔生成答案

In [ ]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = ChatPromptTemplate.from_messages(
      [
          ("human", """你是一個問答任務的助手。使用以下檢索的上下文來回答問題。如果你不知道答案，就直說不知道。最多使用三個句子，保持答案簡潔。

Question: {question}

Context: {context}

Answer:
"""),
      ]
)

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# # Run
generation = rag_chain.invoke({"context": cleaned_docs_format, "question": question})
print(generation)

The agent memory module is a long-term memory database that records agents' experiences in natural language. It includes observations and events provided by the agent, which can trigger new natural language statements through inter-agent communication. The retrieval model surfaces context based on relevance, recency, and importance to inform the agent's behavior.


## 幻覺評分器
這個組件用於評估生成的答案是否基於事實：

In [ ]:
### Hallucination Grader

# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(description="Answer is grounded in the facts, 'yes' or 'no'")

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """你是一個評分員，評估 LLM 生成的內容是否基於/支持於一組檢索的事實。 \n
     給出二元評分 'yes' 或 'no'。'Yes' 表示答案基於/支持於這組事實。"""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "事實集：\n\n {documents} \n\n LLM 生成：{generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": cleaned_docs_format, "generation": generation})

GradeHallucinations(binary_score='yes')

## 答案評分器
用於評估生成的答案是否解決了問題：

In [ ]:
### Answer Grader

# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(description="Answer addresses the question, 'yes' or 'no'")

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """你是一個評分員，評估答案是否解決/回答了問題。\n
給出二元評分 'yes' 或 'no'。'Yes' 表示答案解決了問題。"""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "用戶問題：\n\n {question} \n\n LLM 生成：{generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question,"generation": generation})

GradeAnswer(binary_score='yes')

## 查詢優化器
這個組件用於改寫用戶問題，使其更適合檢索：

In [ ]:
### Question Re-writer

# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Prompt
system = """你是一個問題重寫器，將輸入問題轉換為更適合向量存儲檢索的版本。\n
分析輸入並嘗試理解其底層語義意圖/含義。"""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "這是初始問題：\n\n {question} \n 制定一個改進的問題。"),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'改進後的問題：\n\n如何有效地管理代理人的記憶？'

In [ ]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

## 定義圖的狀態

In [ ]:
#################
#langgraph code
from typing_extensions import TypedDict
from typing import List

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """
    question : str
    generation : str
    documents : List[str]


## 定義節點函數

接下來，我們定義了幾個關鍵的節點函數，包括檢索、生成、文檔評分、查詢轉換和網絡搜索。這些函數將在我們的圖中作為節點使用。

In [ ]:
## graph flow
from langchain.schema import Document

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---[檢索文件]---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)

    # docs = retriever.get_relevant_documents(question)
    # documents = [doc.page_content for doc in docs]

    # docsNEW = [doc.page_content for doc in docs]  # Adjust if the actual attribute name differs
    print("TAKING DICSNEW BRO...>>>>>>>>")

    return {"documents": cleaned_docs_format, "question": question}

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---[生成答案]---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---[檢查文檔與查詢相關性]---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("---[評分]: 文檔相關---")
            filtered_docs.append(d)
        else:
            print("---[評分]: 文檔不相關---")
            continue
    return {"documents": filtered_docs, "question": question}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---[重寫查詢]---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---[網路檢索]---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


## 定義流程控制函數
我們還定義了一些流程控制函數，用於決定下一步應該執行哪個節點。這些函數包括問題路由、生成決策和生成結果評估。

In [ ]:
### Edges ###

def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---[問題分類]---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == 'web_search':
        print("---[問題分類->網路檢索]---")
        return "web_search"
    elif source.datasource == 'vectorstore':
        print("---[問題分類->RAG]---")
        return "vectorstore"

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"




## 建置與編譯整張圖

最後，我們使用 Langgraph 的 StateGraph 來構建我們的適應性 RAG 系統：


In [ ]:
## grapj build
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae
workflow.add_node("transform_query", transform_query) # transform_query

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

## 與圖進行互動

現在我們可以使用編譯後的圖來處理各種查詢。以下是幾個示例：


### 案例一、NFL 選秀相關問題

In [ ]:
from pprint import pprint

# Run
inputs = {"question": "What player at the Bears expected to draft first in the 2024 NFL draft?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node//
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---[問題分類]---
---[問題分類->網路檢索]---
---[網路檢索]---
"Node 'web_search':"
'\n---\n'
---[生成答案]---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('Caleb Williams from USC was expected to be drafted first by the Bears in the '
 '2024 NFL draft.')


In [ ]:
# cleaned_docs_format

### 案例二、生成式代理相關問題

In [ ]:
from pprint import pprint

# Run
inputs = {"question": "什麼是 Agent ?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node//
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---[問題分類]---
---[問題分類->RAG]---
---[檢索文件]---
TAKING DICSNEW BRO...>>>>>>>>
"Node 'retrieve':"
'\n---\n'
---[檢查文檔與查詢相關性]---
---[評分]: 文檔相關---
---[評分]: 文檔相關---
---[評分]: 文檔相關---
---[評分]: 文檔相關---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---[生成答案]---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('在這個上下文中，Agent 是指一種由 GPT-3.5 '
 '驅動的代理人，用於執行簡單任務和互動應用程序。這些代理人結合了語言模型、記憶、規劃和反思機制，以便根據過去的經驗行為，並與其他代理人互動。代理人還可以調用外部 '
 'API 以獲取缺失的信息。')


### 案例三、美國總統相關問題

In [ ]:
from pprint import pprint

# Run
inputs = {"question": "誰是美國建國總統？"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node//
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---[問題分類]---
---[問題分類->網路檢索]---
---[網路檢索]---
"Node 'web_search':"
'\n---\n'
---[生成答案]---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
'喬治·華盛頓是美國建國總統。'


# 結論
通過這個教程，我們展示了如何使用 Langgraph 構建一個適應性 RAG 系統。這個系統能夠根據問題的性質選擇適當的信息來源，評估檢索結果的相關性，並在需要時優化查詢或生成答案。這種方法大大提高了問答系統的靈活性和準確性。